In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install cdlib
import sys
!{sys.executable} -m pip install python-igraph
!{sys.executable} -m pip install leidenalg

     |████████████████████████████████| 153kB 4.4MB/s 
     |████████████████████████████████| 40.6MB 103kB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 1.8MB 31.4MB/s 
     |████████████████████████████████| 24.2MB 1.3MB/s 
  Created wheel for python-louvain: filename=python_louvain-0.14-cp37-none-any.whl size=9292 sha256=1b540414e0664113ca23650f013469d5c2e407858cb78baf8352a0412fc21335
  Stored in directory: /root/.cache/pip/wheels/e7/8d/24/6b3a464bb23e96ecba3f68868e85721534fd8158a9cd7b426b
  Created wheel for karateclub: filename=karateclub-1.0.24-cp37-none-any.whl size=94202 sha256=81a3ea43c99cdbe2f805c3da9a921201b3d825075f2afc6afbe53bc2156dd716
  Stored in directory: /root/.cache/pip/wheels/ab/a4/81/a2761ff51bb1caa318e69e4a30f5d7d39c14f10efe73365279
Successfully built python-louvain karateclub
  Found existing installation: python-louvain 0.15
    Uninstalling python-louvain-0.15:
      Successfully uninstalled python-louvain-0.15

In [3]:
import pandas as pd
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
from cdlib import algorithms
import numpy as np
import pickle
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from tqdm.autonotebook import tqdm
import pickle

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [4]:
import tweepy
from PIL import Image
import requests
from io import BytesIO
consumer_key = "CtmrTSUsnIRHKSgj4ktqK4NKb"
consumer_secret = "1MNfKmptTsrJcq7WMYxOoVwEgFCK5DNMQeC43IZDkPucPObCnZ"
access_key = "1231240089600057344-rqYnkeVvMaB1XwfvrERfI7aF38r69L"
access_secret = "KmRfAoexHxfkHKJaC3hwB0sgtfocyz58IqdzYiWeaXSGO"

In [5]:
path = "/content/drive/My Drive/"
project_name="2_TwitterFollowGraph"
dataframe = pd.read_csv(path+project_name+"/Datasets/Twitter/twitter-final.csv")
columns=list(dataframe.columns)

In [6]:
dataset = pd.read_csv(path+project_name+"/Datasets/Twitter/twitter-2010-ids.csv")
node_id_list = dataset['node_id'].to_list()
twitter_id_list = dataset['twitter_id'].to_list()
node_id_twitter_id = {node_id_list[i]: twitter_id_list[i] for i in range(len(node_id_list))}

In [7]:
def createGraph(dataframe,columns):
  edgelist = dataframe[columns].values.tolist()
  graph = nx.Graph()
  graph.add_edges_from(edgelist)
  return graph

In [8]:
graph = createGraph(dataframe,columns)

In [ ]:
def sortedCommunities(communities):
  communitiesSorted = sorted(communities,key=lambda x:len(x),reverse=True)
  return communitiesSorted

In [ ]:
def topDegreeNodeCommunities(communities,graph,size):
  degree = graph.degree()
  newCommunityList = []
  for community in communities:
    newCommunity = sorted(community,key=lambda x:degree[x],reverse=True)
    newCommunityList.append(newCommunity[:size])
  return newCommunityList

In [ ]:
def generateCommunities(graph,method):
  if method == "leiden":
    communities = algorithms.leiden(graph)
  elif method == "walktrap":
    communities = algorithms.walktrap(graph)
  elif method == "surprise_communities":
    communities = algorithms.surprise_communities(graph)
  communitiesSorted = sortedCommunities(communities.communities)[:20]
  topNodeCommunities = topDegreeNodeCommunities(communitiesSorted,graph,15) # Set third parameter to get required size
  return topNodeCommunities

In [15]:
# Function to extract usernames
def get_username(twitter_id):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    try:
      username = api.get_user(id=twitter_id)
      user_dict = username.__getattribute__("_json")
      return user_dict
    except Exception:
      pass

In [ ]:
def filter_communities(communities_list):
  infile = open(path+project_name+"/PickleFiles/node_info_pickle.pkl",'rb')
  node_map_dict = pickle.load(infile)
  infile.close()
  community_names_list=[[('NA',False)]*len(communities_list[0]) for i in range(len(communities_list))]
  for i,community in enumerate(communities_list):
    for j,member in enumerate(community):
      if member in set(list(node_map_dict)) and node_map_dict[member] is not None:
        community_names_list[i][j]=(node_map_dict[member]['name'],node_map_dict[member]['verified'],node_map_dict[member]['followers_count'])
      else:
        profile_dict=get_username(node_id_twitter_id[member])
        if profile_dict is not None:
          community_names_list[i][j]=(profile_dict['name'],profile_dict['verified'],profile_dict['followers_count'])

  final_community_list=[]
  for community in community_names_list:
    temp_list=[]
    for member in community:
      if member[1] is True:
        temp_list.append((member[0],member[2]))
    if len(temp_list)!=0:
      final_community_list.append(temp_list)
  return final_community_list

In [9]:
# Function to extract usernames
def profile_pic_fetcher(twitter_id):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    try:
      username = api.get_user(id=twitter_id)
      user_dict = username.__getattribute__("_json")
      user_name = user_dict["name"]
      profile_pic_url=user_dict["profile_image_url_https"]
      if "default_profile_normal" not in profile_pic_url:
        profile_pic_url=profile_pic_url.replace("_normal","")
      response = requests.get(profile_pic_url)
      profile_pic = Image.open(BytesIO(response.content))
      return profile_pic,user_name
    except Exception:
      pass

In [10]:
communities_dict={}
page_rank_dict={}

In [ ]:
#Leiden community algorithm
communities_list=generateCommunities(graph,"leiden")
final_communities_list=filter_communities(communities_list)
communities_dict["leiden"]=final_communities_list
print(final_communities_list)

[[('CNN Breaking News', 61190864), ('ashton kutcher', 17622863), ('Ellen DeGeneres', 79150582), ('Britney Spears', 56074625), ('Twitter', 59475201), ('Barack Obama', 130091376), ('Lance Armstrong', 3225007), ('Oprah Winfrey', 43646337), ('Ryan Seacrest', 15390398), ('Demi Moore', 4565235), ('SHAQ', 15628657), ('The New York Times', 49711344), ('jimmy fallon', 51868620), ('Al Gore', 3054622)], [('Leo Laporte, Chief TWiT and The Tech Guy', 532306)], [('Stephen Fry', 12634103), ('Jonathan Ross', 5044515), ('Russell Brand', 11206944), ('Phillip Schofield', 4564965), ('Chris Moyles', 3600198)], [('Selena Gomez', 64958647), ('Demi Lovato', 55285517), ('Jonas Brothers', 4827694)], [('Josh Olin', 103247)], [('Luciano Huck', 13283352)], [('Stocktwits', 877136)], [('little farma', 564661), ('Superinteressante', 4038373), ('Radio-Canada Techno', 15129)], [('Glenn Beck', 1312959), ('Brad Paisley', 4367501)]]


In [ ]:
#Surprise communities algorithm
communities_list=generateCommunities(graph,"surprise_communities")
final_communities_list=filter_communities(communities_list)
communities_dict["surprise_communities"]=final_communities_list
print(final_communities_list)

[[('CNN Breaking News', 61190864), ('ashton kutcher', 17622863), ('Ellen DeGeneres', 79150582), ('Britney Spears', 56074625), ('Twitter', 59475201), ('Lance Armstrong', 3225007), ('Oprah Winfrey', 43646337), ('Ryan Seacrest', 15390398), ('Demi Moore', 4565235), ('SHAQ', 15628657), ('The New York Times', 49711344), ('jimmy fallon', 51868620), ('Al Gore', 3054622), ('Coldplay', 23301988)], [('Stephen Fry', 12634103), ('Jonathan Ross', 5044515), ('Russell Brand', 11206944), ('Phillip Schofield', 4564965), ('Chris Moyles', 3600198), ('The Unnamed Artist', 5762042), ('Jimmy Carr', 6737846), ('fearne cotton', 6890318), ('Jamie Oliver', 6516271)], [('George Mason University', 27191)], [('buten un binnen', 39868)], [('elizabeth', 4383)]]


In [ ]:
#Walktrap communites algorithm
communities_list=generateCommunities(graph,"walktrap")
final_communities_list=filter_communities(communities_list)
communities_dict["walktrap"]=final_communities_list
print(final_communities_list)

[[('CNN Breaking News', 61190864), ('Britney Spears', 56074625), ('Twitter', 59475201)], [('ashton kutcher', 17622863), ('Ellen DeGeneres', 79150582), ('Lance Armstrong', 3225007), ('Oprah Winfrey', 43646337), ('Demi Moore', 4565235), ('The New York Times', 49711344), ('jimmy fallon', 51868620), ('Al Gore', 3054622), ('@bbcclick', 2152173), ('Penn Jillette', 1777615)], [('Selena Gomez', 64958647), ('Demi Lovato', 55285517)], [('Leo Laporte, Chief TWiT and The Tech Guy', 532306), ('John C. Dvorak', 99736), ('Alex Albrecht', 77584), ('Adam Savage', 1473338)], [('Phillip Schofield', 4564965), ('Jimmy Carr', 6737846), ('fearne cotton', 6890318)], [('Fox News', 20146730), ('Glenn Beck', 1312961)], [('Deepak Chopra', 3257759)], [('Jan Willem Alphenaar', 18699), ('buten un binnen', 39865)], [('Smashing Magazine', 963478), ('Google AdSense', 565352)]]


In [11]:
def rank_on_page_rank(graph):
  pagerank_result = dict(pagerank(graph))
  pagerank_result = sorted(pagerank_result.items(),key = lambda x:x[1],reverse=True)
  return pagerank_result

In [12]:
pagerank_result=rank_on_page_rank(graph)

In [13]:
def filter_pagerank(pagerank_result,size):
  final_pagerank_result=[]
  for entity in tqdm(pagerank_result[:size]):
    profile_dict=get_username(node_id_twitter_id[entity[0]])
    if profile_dict is not None and profile_dict['verified']==True:
        final_pagerank_result.append((profile_dict['name'],profile_dict['followers_count'],node_id_twitter_id[entity[0]]))
  return final_pagerank_result

In [16]:
final_pagerank_result=filter_pagerank(pagerank_result.copy(),100)
page_rank_dict["pagerank"]=final_pagerank_result

In [17]:
final_pagerank_result

[('CNN Breaking News', 61257398, 428333),
 ('ashton kutcher', 17609356, 19058681),
 ('Ellen DeGeneres', 79110794, 15846407),
 ('Barack Obama', 130198925, 813286),
 ('Britney Spears', 56081171, 16409683),
 ('Twitter', 59529056, 783214),
 ('Oprah Winfrey', 43657140, 19397785),
 ('Lance Armstrong', 3222482, 16727535),
 ('Ryan Seacrest', 15379296, 16190898),
 ('SHAQ', 15647369, 17461978),
 ('Al Gore', 3052245, 17220934),
 ('The New York Times', 49806056, 807095),
 ('Demi Moore', 4561994, 19554706),
 ('Coldplay', 23294661, 18863815),
 ('jimmy fallon', 51870114, 15485441),
 ('NPR Politics', 3077586, 5741722),
 ('Stephen Fry', 12632770, 15439395),
 ('Google', 22958396, 20536157)]

In [ ]:
filename = open(path+project_name+"/PickleFiles/communities.pkl","wb")
pickle.dump(communities_dict, filename)
filename.close()

In [18]:
filename = open(path+project_name+"/PickleFiles/pagerank.pkl","wb")
pickle.dump(page_rank_dict, filename)
filename.close()

In [19]:
for pagerankResult in final_pagerank_result:
  profile_pic,username=profile_pic_fetcher(pagerankResult[2])
  profile_pic.save(path+project_name+"/PageRankImages/Twitter/{}.png".format(username))